In [1]:
import numpy as np

In [8]:
import numpy as np

def lbfgs(f, grad_f, x0, m=10, tol=1e-5, max_iter=1000):
    
    x = x0.copy()
    k = 0  # iteration counter

    s_list = []   
    y_list = []   
    rho_list = [] 

    g = grad_f(x)
    BOUND = 0  # number of updates used

    while np.linalg.norm(g) > tol and k < max_iter:
        q = g.copy()
        alpha = []

        if k < m:
            INCR = 0
            BOUND = k
        else:
            INCR = k - m
            BOUND = m

        for i in range(BOUND - 1, -1, -1):
            j = i + INCR
            if j >= len(s_list):  # safety check
                continue
            s = s_list[j]
            y = y_list[j]
            rho = rho_list[j]
            alpha_i = rho * np.dot(s, q)
            alpha.append(alpha_i)
            q = q - alpha_i * y

        alpha = alpha[::-1]  # to use in forward loop later

        if k > 0:
            y_last = y_list[-1]
            s_last = s_list[-1]
            gamma = np.dot(s_last, y_last) / np.dot(y_last, y_last)
        else:
            gamma = 1.0
        r = gamma * q

        for i in range(0, BOUND):
            j = i + INCR
            if j >= len(s_list):  # safety check
                continue
            s = s_list[j]
            y = y_list[j]
            rho = rho_list[j]
            beta = rho * np.dot(y, r)
            r = r + s * (alpha[i] - beta)

        #search direction and backtracking line search
        p = -r
        t = 1.0
        while f(x + t * p) > f(x) + 1e-4 * t * np.dot(g, p):
            t *= 0.5
            if t < 1e-10:
                break

        #position and gradient update
        x_new = x + t * p
        g_new = grad_f(x_new)
        s_k = x_new - x
        y_k = g_new - g

        #store s_k, y_k, and ρ_k if curvature condition satisfied
        if np.dot(s_k, y_k) > 1e-10:
            s_list.append(s_k)
            y_list.append(y_k)
            rho_list.append(1.0 / np.dot(y_k, s_k))

            #maintaining memory size
            if len(s_list) > m:
                s_list.pop(0)
                y_list.pop(0)
                rho_list.pop(0)

        x = x_new
        g = g_new
        k += 1

        print(f"Iter {k:2d} | f(x) = {f(x):.6f} | ||grad|| = {np.linalg.norm(g):.2e}")

    return x,f(x), k  


In [ ]:
def rosenbrock(x):
    return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

In [32]:
#z=(x-3)**2 + (y-2)**2
def f8(v):
    x, y = v
    return (x - 3)**2 + (y - 2)**2

In [12]:
#z=5x**2 + y**2
def f5(v):
    x, y = v
    return 5 * x**2 + y**2

In [36]:
#z=sin(x)+y**2
def f10(v):
    x, y = v
    return np.sin(x) + y**2

In [16]:
def f2(v):
    x, y = v
    return np.exp(x) + np.exp(y) + x**2 + y**2

In [ ]:
#creating a noisy function
K = 10
alpha = 0
np.random.seed(27)
phi = 1 * np.pi * np.random.rand(K, K)  # fixed phases

# Function f(v) = sin(x) + y^2 + noise
#changed to f(v) = x^2 + y^2 + noise
def f_structured(v):
    x, y = v
    f_val = x**2 + y**2
    for k1 in range(K):
        for k2 in range(K):
            amp = 1.0 / (1 + k1**2 + k2**2)**(alpha / 2)
            f_val += 0.05*(amp * np.cos(k1 * x + k2 * y + phi[k1, k2]))
    return f_val


In [4]:
#writing a function grad that calculates gradient numerically 
def grad_numerical(func, v, h=1e-7):
    grad = np.zeros_like(v)
    for i in range(len(v)):
        v_plus = np.copy(v)
        v_minus = np.copy(v)
        v_plus[i] += h
        v_minus[i] -= h
        grad[i] = (func(v_plus) - func(v_minus)) / (2 * h)
    return grad

In [ ]:
x0 = np.array([10, -10.0])
xmin,fval,itera = lbfgs(f5, lambda x: grad_numerical(f5, x), x0)
print("\nMinimum found at:", xmin)
print("Function value at minimum:", fval)
print("Iterations:", itera)

Iter  1 | f(x) = 87.499999 | ||grad|| = 2.92e+01
Iter  2 | f(x) = 39.859709 | ||grad|| = 1.28e+01
Iter  3 | f(x) = 17.727278 | ||grad|| = 9.30e+00
Iter  4 | f(x) = 0.801095 | ||grad|| = 3.43e+00
Iter  5 | f(x) = 0.000006 | ||grad|| = 1.06e-02
Iter  6 | f(x) = 0.000000 | ||grad|| = 3.99e-04
Iter  7 | f(x) = 0.000000 | ||grad|| = 1.31e-05
Iter  8 | f(x) = 0.000000 | ||grad|| = 2.66e-07

Minimum found at: [ 2.52456323e-08 -4.23564890e-08]
Function value at minimum: 4.9807819035378685e-15
Iterations: 8


: 